This code seeks to identify what the best way to optimize a listing on Neighbor.com is. We seek to identify specific features that are correlated with reservation status. The project is organized as follows
1. Import Libraries
2. Pull in Data
3. Clean Data
4. Feature Engineering/Deal with Missing Values
5. Model

#  1. Import Libraries

In [271]:
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy import stats
from dateutil.relativedelta import relativedelta
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
pd.set_option('display.max_columns', None)

from scipy.stats import norm
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, classification_report, accuracy_score, mean_squared_error
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from imblearn.over_sampling import SMOTE
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})

# 2. Pull in Data

These strings in red need to be replaced with the path of the corresponding files on your computer. These files should be updated periodally by requesting them from preferably Colton Gardner or anyone who can use SQL to pull data from the website.

In [272]:
# Read in Data Tables

# listings.csv
listings = pd.read_csv('C:\\Users\\ben1c\\Projects\\Neighbor\\Listing Optimization\\Regression Data\\listings.csv')
# users.csv
users = pd.read_csv('C:\\Users\\ben1c\\Projects\\Neighbor\\Listing Optimization\\Regression Data\\users.csv')
# reservations.csv
reservations = pd.read_csv('C:\\Users\\ben1c\\Projects\\Neighbor\\Listing Optimization\\Regression Data\\reservations.csv')
# risting_metadata.csv
listingmeta = pd.read_csv('C:\\Users\\ben1c\\Projects\\Neighbor\\Listing Optimization\\Regression Data\\listing_metadata.csv')
# photos.csv
photos = pd.read_csv('C:\\Users\\ben1c\\Projects\\Neighbor\\Listing Optimization\\Regression Data\\photos.csv')
# user_metadata.csv
usermeta = pd.read_csv('C:\\Users\\ben1c\\Projects\\Neighbor\\Listing Optimization\\Regression Data\\user_metadata.csv')

In [273]:
# Drop all duplicates, deleted posts or any that aren't published
listings.drop_duplicates()
listings = listings[listings.status == 'Published']
listings = listings[listings['deleted'] != 'True']

# Return user data from hosts only
users = users[users['is_host']==True]

The following cell deletes all listings from the last thirty days because they are not representative of the sample we are running this regression on.

In [274]:
# Delete listings made in the last 30 days
listings = listings[pd.to_datetime(listings['created_at']) <= pd.datetime.now().date()-relativedelta(months=1)]

# 3. Clean Data

In [275]:
# Creates new column with Description Length Boolean
listings['Description Length'] = np.where(listings['summary'].str.len()>=30, '1', '0')

# Creates new column with Title Length Boolean
listings['Title Length'] = np.where((listings['listing_name'].str.len()>=10), 1, 0)

# Create Storage Type Category
listings['Storage Type'] = listings['storage_type'].values

# Create Accessibility Category
listings['Accessibility'] = listings['access'].values

# Create Price Per Square Ft
listings['Price Per SQFT'] = (listings['monthly_price'].values)/(listings['width'].values * listings['length'])

# Create Day Posted
listings['Day Posted'] = pd.to_datetime(listings['created_at']).dt.dayofweek

# Create Month Posted
listings['Month Posted'] = pd.to_datetime(listings['created_at']).dt.month

# Create Reserved Status
listings = listings.assign(Reserved=listings.id.isin(reservations.listing_id).astype(int))

In [276]:
#USER METADATA

#picking relevant columns
dfusermeta = pd.DataFrame(usermeta, columns=['id','user_id','key','value'])
df12 = pd.DataFrame(usermeta, columns=['id','user_id','key','value'])

#New column with superhost level inputted
dfusermeta['Superhost Level'] = dfusermeta[dfusermeta['key']=='superhost_level']['value'].astype(int)
df12['Superhost Level'] = np.where(dfusermeta['key']=='superhost_level',dfusermeta['value'],'0')
df123 = df12.loc[df12['Superhost Level']!='0']
users = pd.merge(users, df123[['user_id', 'Superhost Level']], left_on='id', right_on='user_id', how='left')


#Pull in enterprise status
dfusermeta['Enterprise'] = dfusermeta[dfusermeta['key']=='enterprise_account']['value']
df12['Enterprise'] = np.where(dfusermeta['key']=='enterprise_account', '1', '0')
df124 = df12.loc[df12['Enterprise']!='0']
users = pd.merge(users, df124[['user_id', 'Enterprise']], left_on='id', right_on='user_id', how='left')

In [277]:
#LISTING METADATA
df1 = pd.DataFrame(listingmeta, columns= ['id','listing_id','key','value',])
df23 = pd.DataFrame(listingmeta, columns= ['id','listing_id','key','value',])

# 4. Feature Engineering
### Fix Price Score/Manual Score

Price score ranges from 0 to 1 with .5 being the best score. We manipulated price score such that its range is from -.5 to 0 where a value of 0 represents the best possible score and a value of -.5 means that the assigned price score was .5 away form the ideal price score of .5. Therefore, if you were given a score of .7 (your price is higher than our recommendation) then the value in the column would read in as -.2 because you were .2 away from the ideal price score. We did this to make the coefficient on price score easier to understand.

In [278]:
#Pull price score data
df23['Price Score'] = np.where(df1['key']=='price_score',df1['value'],'0')
df234 = df23.loc[df23['Price Score']!='0']
df234['Price Score'] = -(abs(df234['Price Score'].astype(float) + -.5))

listings = pd.merge(listings, df234[['listing_id', 'Price Score']], left_on='id', right_on='listing_id', how='left')

#First Month Discount
df23['Discount'] = np.where(df1['key']=='first_month_discount',df1['value'],'0')
df234 = df23.loc[df23['Discount']!='0']
listings = pd.merge(listings, df234[['listing_id', 'Discount']], left_on='id', right_on='listing_id', how='left')

#Pull manual score data
df23['Manual Score'] = np.where(df1['key']=='manual_score',df1['value'],'0')
df234 = df23.loc[df23['Manual Score']!='0']
listings = pd.merge(listings, df234[['listing_id', 'Manual Score']], left_on='id', right_on='listing_id', how='left')

In [279]:
#Pull feature data
dffeat = pd.DataFrame(listingmeta, columns= ['id','listing_id','key','value'])
dffeat = dffeat[dffeat['key']=='features']
dffeat.drop('key', axis = 1, inplace=True)

dffeat1 = dffeat['value'].str.split(pat=",",expand=True)      
dffeat1 = pd.get_dummies(dffeat1.apply(pd.Series).stack()).sum(level=0)
dffeat1['id'] = dffeat['listing_id']

Some of the features are listed twice in the original dataframe so we will replace any values of 2 with a 1

In [280]:
dffeat1.replace(2, 1, inplace=True)

In [281]:
listings = pd.merge(listings, dffeat1, on='id', how='left')

In [282]:
#USERS
#picking relevant columns
dfusers = pd.DataFrame(users, columns= ['id','bio','is_host','profile_photo_id' , 'lead_source'])

#return user data from hosts only
dfusers = dfusers[dfusers['is_host']==True]

#if they have a bio, counts the number of characters
dfusers['bio length'] = np.where(dfusers['bio'].isna(), 0, 1)

#Add new column, if they have profile pic then '1' else '0'
dfusers['Profile Photo'] = np.where(dfusers['profile_photo_id'].notnull().values,'1','0')

In [283]:
#PHOTOS

#picking relevant columns
dfphoto = pd.DataFrame(photos, columns= ['id','imageable_type','imageable_id','filename', 'deleted'])
dfphoto2 = pd.DataFrame(photos, columns= ['id','imageable_type','imageable_id','filename', 'deleted'])

#returning only undeleted listing photo data
dfphoto = dfphoto[dfphoto['imageable_type']=='Listing']

#insert new column for streetview and label with 1 and 0 for rows with streetview
dfphoto2['streetview'] = np.where((dfphoto2['filename']=='streetView') |(dfphoto2['filename']=='streetview'),'1','0')


dfphoto3 = dfphoto2.loc[dfphoto2['streetview']=='1']
dfphoto3 = dfphoto3[dfphoto3['deleted']==False]

#Add streetview count on listing df
listings = pd.merge(listings, dfphoto3[['imageable_id','streetview']], left_on='id', right_on='imageable_id', how='left')


#count number of pictures for a listing
dfphoto['Frequency'] = dfphoto.groupby('imageable_id')['imageable_id'].transform('count')

#drop duplicate rows for the same listing
dfphoto.drop_duplicates('imageable_id', inplace=True)
dfphoto = dfphoto[dfphoto['deleted']==False]

#Add number of pictures on a listing to listing df
listings = pd.merge(listings, dfphoto[['imageable_id','Frequency']], left_on='id', right_on='imageable_id', how='left')

In [284]:
# Merge the listing and user data together
listings = pd.merge(listings,
                    users,
                    left_on='user_id',
                    right_on='id',
                    how='left'
                     )

In [285]:
finaldf = pd.DataFrame()

# Bring in Listing Info
finaldf['Listing-ID'] = listings['id_x']
finaldf['Accessibility'] = listings['Accessibility']
finaldf['Storage Type'] = listings['Storage Type']
finaldf['Title Length'] = listings['Title Length']
finaldf['Description Length'] = listings['Description Length']
finaldf['Indoor'] = listings['indoor']
finaldf['Day Posted'] = listings['Day Posted']
finaldf['Month Posted'] = listings['Month Posted']
finaldf['Photos of the Space'] = listings['Frequency']
finaldf['Streetview'] = listings['streetview']
finaldf['Discount'] = listings['Discount']

# Bring in Host Info
finaldf['Bio Included'] = dfusers['bio length']
finaldf['Profile Photo'] = dfusers['Profile Photo']
finaldf['Phone Verified'] = listings['phone_verified']
finaldf['Photo ID Verified'] = listings['photo_id_verified']
finaldf['Superhost Level'] = listings['Superhost Level']
finaldf['Camera'] = listings['camera']
finaldf['Climate Controlled'] = listings['climate_controlled']
finaldf['Lockable'] = listings['is_lockable']
finaldf['No Pets'] = listings['no_pets']
finaldf['No Stairs'] = listings['no_stairs']
finaldf['Private Entrance'] = listings['private_entrance']
finaldf['Smoke Detectors'] = listings['smoke_detectors']
finaldf['Smoke Free'] = listings['smoke_free']
finaldf['Enterprise'] = listings['Enterprise']

# Bring in Review info
finaldf['Manual Score'] = listings['Manual Score']
finaldf['Price Score'] = listings['Price Score']

# Add Reserved Info
finaldf['Reserved'] = listings['Reserved']

#### To remove the entreprise or peer to peer listings simply uncomment the respective lines. To regress on everything just comment out this entire cell

In [286]:
# Regress on Enterprise Listings
#finaldf = finaldf.loc[finaldf['Enterprise']=='1']

# Regress on p2p Listings
#finaldf = finaldf.loc[(finaldf['Enterprise']!='1')]

#Unused Variables
#finaldf['Price Per SQFT'] = listings['Price Per SQFT']

In [287]:
finaldf.drop_duplicates(inplace=True)

# Cleaning missing values
#### The following are our reasoning on filling these missing values with zeroes

Discount-maybe it didn't exist in the beginning so some of the early ones are are NaN, replace t/f with TRUE/FALSE, then Fill with ZERO  
Features - 108 are missing for them all and it appears that they are the same listings, seems like they are outside so it probably means that they just don't have any of those features. FILL WITH ZERO  
Price Score- I don't see anything special. DROP ROW  
Manual Score- I don't see anything special. DROP ROW  
Phone Verified- Fill with zeros  
Superhost level - FIll with Zeros  
Bio Length- Fill with Zero  
Photos of Space- FIll with Zero, because there are no zero values  
Streetview- Fill with 0 becuase they don't have it.  

In [288]:
finaldf = finaldf.fillna(0)

In [289]:
finaldf.replace('true', 1, inplace=True)
finaldf.replace('false', 0, inplace=True)
finaldf.replace('True', 1, inplace=True)
finaldf.replace('False', 0, inplace=True)
finaldf.replace('TRUE', 1, inplace=True)
finaldf.replace('FALSE', 0, inplace=True)
finaldf.replace('f', 0, inplace=True)
finaldf.replace('t', 1, inplace=True)
finaldf.replace(False, 0, inplace=True)
finaldf.replace(True, 1, inplace=True)

In [290]:
dummies = ['Storage Type', 'Accessibility']
finaldf = pd.get_dummies(finaldf, columns=dummies)

# 5. Modeling

There are many types of regressions that you could run here. Essentially in the next two cells you may drop whatever variables you want to pick what you want to regress on reservations

In [291]:
d = ['Listing-ID', 'Indoor', 'Day Posted', 'Month Posted', 
     'Superhost Level', 'Manual Score', 'Photo ID Verified', 'Enterprise']

c = ['Camera', 'Climate Controlled', 'Lockable',
       'No Pets', 'No Stairs', 'Private Entrance', 'Smoke Detectors', 'Smoke Free', 'Reserved']

control = ['Streetview', 'Discount', 'Price Score', 'Title Length', 'Bio Included', 
           'Photos of the Space', 'Description Length', 'Profile Photo', 'Reserved']

dumstorage = [col for col in finaldf if col.startswith('Storage Type')]  + ['Reserved']
dumaccess = [col for col in finaldf if col.startswith('Accessibility')]
dum = dumstorage + dumaccess


#Regress on the features that a space has
def featuresreg(df):
    X = pd.DataFrame(df, columns=c)
    y = X['Reserved']
    X = X.drop(['Reserved'], axis=1)
    return X, y

#Regress on all the data
def alldatareg(df):
    X = df
    y = X['Reserved']
    X = X.drop(['Reserved'], axis=1)
    return X , y

#Regress on Storage Type
def storagetypereg(df):
    X = pd.DataFrame(df, columns=dumstorage)
    y = X['Reserved']
    X = X.drop(['Reserved'], axis=1)

    return X, y

# Regress on factors that we can influence
def factorswecancontrol(df):
    X = pd.DataFrame(df, columns=control)
    y = X['Reserved']
    X = X.drop(['Reserved'], axis=1)
    return X, y


In [299]:
X, y = alldatareg(finaldf)
X, y = featuresreg(finaldf)
X, y = storagetypereg(finaldf)
X, y = factorswecancontrol(finaldf)

1    446
0    282
Name: Discount, dtype: int64

## How is Y distributed?

In [1]:
sns.distplot(y)

NameError: name 'sns' is not defined

The following block should only be run if "y" is imbalanced

In [294]:
# Synthetic Resampling for unbalanced data
X_resampled, y_resampled = SMOTE(random_state=12).fit_resample(X, y)
# # Scale the Data
X_train, X_test, y_train, y_test = train_test_split(StandardScaler().fit_transform(X_resampled), y_resampled, test_size=0.33, random_state=42)

If the reserved rate is balanced then run this block. But don't run both.

In [295]:
#X_train, X_test, y_train, y_test = train_test_split(StandardScaler().fit_transform(X), y, test_size=0.33, random_state=42)

ValueError: Expected 2D array, got 1D array instead:
array=[1. 0. 0. ... 0. 0. 0.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [298]:
clf = linear_model.LogisticRegressionCV(penalty='l2', cv=3)
clf.fit(X_train, y_train)
train_predictions = clf.predict(X_train)
test_predictions = clf.predict(X_test)
print("Train acc: {}".format(accuracy_score(y_train, train_predictions)))
print("Test acc: {}".format(accuracy_score(y_test, test_predictions)))
pd.DataFrame(sorted(list(zip(X.columns, clf.coef_[0])), key=lambda x: x[1]), columns={'Feature', 'Coefficient'})

Train acc: 0.775661717236927
Test acc: 0.7942332896461337


,Feature,Coefficient
0,D,-0.337288
1,u,-0.201193
2,c,-0.100216
3,o,0.011265
4,n,0.022767
5,t,0.023752
6,s,0.057278
7,i,0.919796


In [297]:
logresults = pd.DataFrame(sorted(list(zip(X.columns, clf.coef_[0])), key=lambda x: x[1]), columns={'Feature', 'Coefficient'})

text_kwargs = dict(fontsize=14, family='Arial')

fig, ax = plt.subplots(figsize=(14, 4))
ax.bar(logresults['Feature'], logresults['Coefficient'], width=.95, label='Coefficient Value', color='tab:blue')
ax.set_ylim(-1, 1)
ax.set_ylabel('Percentage', **text_kwargs)
ax.set_xlabel('Variable', **text_kwargs)
plt.xticks(rotation=45)
ax.legend(loc='upper right')
fig.suptitle('Logistic Regression Coefficient', fontsize=20, fontweight='bold')

plt.savefig('LRgraph.png')

AttributeError: 'Series' object has no attribute 'columns'

# The cells below are for running a Random Forest Classifier to predict Reservation Status
You will need change the parameters to optimize the accuracy

In [ ]:
# These are the parameters to change. You will need to change them to maximize your accuracy score
n_estimators = [90]
max_depth = [15, 12, 11]
class_weights = ['balanced', None]
best_f1 = 0

for est in n_estimators:
    for depth in max_depth:
        for wgt in class_weights:
            print(est, depth, wgt)
            clf = RandomForestClassifier(n_estimators=est, max_depth=depth, oob_score=True, class_weight=wgt)
            clf.fit(X_train, y_train)
            f1 = f1_score(y_train, np.argmax(clf.oob_decision_function_ , 1))
            if f1 > best_f1:
                best_f1 = f1
                best_params = (est, depth, wgt)

In [ ]:
print("Best OOB F1: {}".format(best_f1))
print("Best params: {}".format(best_params))

In [ ]:
rfc = RandomForestClassifier(n_estimators=best_params[0], max_depth=best_params[1], class_weight=best_params[2])

rfc.fit(X_train, y_train)
test_predictions = rfc.predict(X_test)
print("Test Classification Report:")
print(classification_report(y_test, test_predictions))

In [ ]:
feature_imp = sorted(list(zip(X.columns, rfc.feature_importances_)), key=lambda x: x[1], reverse=True)
pd.Series([x[1] for x in feature_imp[:]], index=[x[0] for x in feature_imp[:]]).plot(kind='bar', figsize=(14, 4))
fig.suptitle('Random Forest Feature Importances', fontsize=20, fontweight='bold')

plt.savefig('RFCgraph.png')

# The cell below will write outputs to a separate document
These include:  
Final dataframe that is being regressed  
Accuracy scores for each regression  
Graphs for each regression  
  
You will need to change the file path to tell it where to write the files. Pick any file name you like.

In [ ]:
# with open('C:\\Users\\ben1c\\Projects\\Neighbor\\Listing Optimization\\Regression Data\\finaldfEnterprise.csv', 'a') as f:
#     finaldf.to_csv(f, header=True)